# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.8 MB/s eta 0:00:00
  Installing b

In [2]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2578 in from_pretrained   │
│                                                                                                  │
│   2575 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2576 │   │   │   │   }                                                                         │
│   2577 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2578 │   │   │   │   │   raise ValueError(                                                     │
│   2579 │   │   │   │   │   │   """                                                               │
│   2580 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2581 │   │   │   │   │   │   the quantized model. If you have set a value for `max_memory` yo  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you have set a value for `max_memory` you should increase that. To 
have
                        an idea of the modules that are set on the CPU or RAM you can print model.hf_device_map.

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())
        return output.split("### Response:")[1].strip()

In [ ]:
import pandas as pd
!pip install html_sanitizer

from google.colab import drive
drive.mount('/content/drive')

# HTML sanitization
from html_sanitizer import Sanitizer
sanitizer = Sanitizer({
    'tags': ('h1', 'h2', 'p'),
    'attributes': {},
    'empty': set(),
    'separate': set(),
})

# Source file

file = pd.read_csv('/content/drive/MyDrive/HWdb1.csv')

len(file)

file = file[file['content.rendered'].notnull()]

len(file)
fileFiltered = file.loc[file['content.rendered'].str.contains("hate speech | minority | Hate speech | Minority")]

fileFiltered

In [ ]:



# DF to save to 
processed = pd.DataFrame(
    columns=['date', 'title.rendered', 'excerpt.rendered', 'link', 'geocoded'])

In [ ]:
instruction= """
You are an excellent linguist. The task is to label location entities in the given sentence down to a village or a city or a state. Below are some examples. 
Respond with a accurate location. The location must be geocodeable and in English.

Example 1:
Input: <p>On February 9, at Baramati, in rural Pune, Hindu Jangarjana Morcha organized by Sakal Hindu Samaj was successfully held where hate offender Kalicharan Maharaj freely delivered his hate speech rife with blatant lies and hysteria. This, despite urgent appeals being made by Citizens for Justice and Peace (CJP) to the DGP, Maharashtra as well as SP of  [&hellip;]</p>
Output: Baramati, Pune,

Example 2:
Input: <p>A 400-year-old chapel in western India faces the threat of demolition as local politicians aim to acquire the land and use it for a football pitch, local Catholics have claimed. They are protesting that the chapel of Our Lady Of Remedies in Daman, built during the Portuguese colonial era in 1607, is threatened by a [&hellip;]</p>
Output: Daman

Example 3:
Input: <p>KOPPAL: A 30-year-old woman was allegedly beaten up with a slipper by a man belonging to an upper caste after her cattle strayed into his agriculture field. The incident took place at Rampur village in Koppal district on February 3. The victim, who is from the Dalit community, said the accused, Amaresh Kumbar, beat her up [&hellip;]</p>
Output: Rampur, Koppal

Input:
"""

import tensorflow as tf
tf.keras.backend.clear_session()
# importing library
import csv
 
# opening the csv file in 'w' mode
ofile = open('/content/geocoded_Alpaca_GCollab.csv', 'a', newline ='')
 

writer = csv.DictWriter(ofile, fieldnames=['date', 'title', 'excerpt', 'link', 'locations'])

for index, row in fileFiltered.iloc[230:].iterrows():
  loc = evaluate(instruction, file.at[index, 'excerpt.rendered'])
  print(loc)
  # Save data to dataframe
  processed = {'date': file.at[index, 'date'], 
               'title': file.at[index, 'title.rendered'], 
               'excerpt': file.at[index, 'excerpt.rendered'], 
               'link': file.at[index, 'link'], 
               'locations': loc
  }
  print(processed)
  # Save to csv every loop
  writer.writerow(processed)
ofile.close
